In [87]:
import cv2
import numpy as np 
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import matplotlib.image as image
from PIL import Image, ImageOps
from numpy import asarray
import threading

In [88]:
mp_holistic = mp.solutions.holistic #Holistic model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities

In [89]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#Color conversion BGR 2 RGB
    image.flags.writeable = False #Image is unwriteable
    results = model.process(image)#Make prediction
    image.flags.writeable = True#Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#color conversion RGB 2 BGR
    return image, results

In [90]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS)#Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)#Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)#Draw hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)#Draw hand connections

In [91]:
def draw_styled_landmarks(image, results):
    #Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness = 1, circle_radius =1),
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness = 1, circle_radius =1)
                             )
    #Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness = 2, circle_radius =2)
                             )
    #Draw hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness = 2, circle_radius =2)
                             )
    #Draw hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness = 2, circle_radius =2)
                             )

In [92]:
cap = cv2.VideoCapture(0)
#Set mediapipe
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret, frame = cap.read()

        #Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        #Draw landmarks
        draw_styled_landmarks(image, results)
        #Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #Break loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [93]:
results.face_landmarks.landmark

[x: 0.8099812269210815
y: 1.0616708993911743
z: -0.029004855081439018
, x: 0.8094944953918457
y: 1.0152993202209473
z: -0.07476149499416351
, x: 0.8107227683067322
y: 1.024168610572815
z: -0.03602856397628784
, x: 0.7991161942481995
y: 0.9401265978813171
z: -0.06132042407989502
, x: 0.8097819089889526
y: 0.9959384799003601
z: -0.08113937824964523
, x: 0.8107519149780273
y: 0.9657268524169922
z: -0.07780826836824417
, x: 0.8143966197967529
y: 0.8859578371047974
z: -0.04561672359704971
, x: 0.7088284492492676
y: 0.8632528185844421
z: 0.012216000817716122
, x: 0.8163906335830688
y: 0.8207470774650574
z: -0.03873748704791069
, x: 0.81702721118927
y: 0.7913717031478882
z: -0.04385483264923096
, x: 0.8209244608879089
y: 0.6987980604171753
z: -0.04013069346547127
, x: 0.8098529577255249
y: 1.0695174932479858
z: -0.025757217779755592
, x: 0.8098740577697754
y: 1.0747824907302856
z: -0.02040659263730049
, x: 0.8100693821907043
y: 1.0759857892990112
z: -0.013857024721801281
, x: 0.81082355976104

In [ ]:
draw_styled_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# Vẽ điểm trên ảnh bất kỳ

In [ ]:
test_img = Image.open("frame1.jpg")
to_array_test_img = asarray(test_img)
to_array_test_img.shape

In [ ]:
plt.imshow(cv2.rotate(to_array_test_img, cv2.ROTATE_180))

In [ ]:
# test_img = test_img.resize((640, 480))

In [ ]:
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    a, b = mediapipe_detection(to_array_test_img, holistic)
    draw_styled_landmarks(a, b)

In [ ]:
draw_styled_landmarks(a, b)

In [ ]:
plt.imshow(cv2.cvtColor(a, cv2.COLOR_BGR2RGB))

# End

In [94]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if \
        results.face_landmarks else np.zeros(468*3)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if \
        results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if \
        results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if \
        results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [95]:
results_test = extract_keypoints(results)

In [96]:
results_test.shape

(1662,)

# setup folder for collection

In [97]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_60FPS')

# Actions that we try to detect# n_inputs = 30, input = 30 frames
actions = np.array(['Walk_', 'Wait_', 'Hello_L','Hello_R','Name_','No_','Sorry_R','Thanks_L',
                    'Fire_','Give up_','House_','Ready_','Drink_','Sleep_','Where_'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [ ]:
# for action in actions: 
# #     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
#     try:
#         os.makedirs(os.path.join(DATA_PATH, action + '_image'))
#     except:
#         pass
#     for sequence in range(no_sequences):
#         try: 
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

# Collect keypoint for training

In [ ]:
# cap = cv2.VideoCapture(0)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     # NEW LOOP
#     # Loop through actions
#     for action in actions:
#         # Loop through sequences aka videos
#         for sequence in range(no_sequences):
#             count = 0
#             # Loop through video length aka sequence length
#             for frame_num in range(sequence_length):

#                 # Read feed
#                 ret, frame = cap.read()
                
#                 # Make detections
#                 image, results = mediapipe_detection(frame, holistic)
# #                 print(results)

#                 # Draw landmarks
#                 draw_styled_landmarks(image, results)
# #                 img_path = os.path.join(DATA_PATH, action + '_image', 'frame%d.jpg' % count)
#                 count+=1
# #                 cv2.imwrite(img_path, image)
#                 # NEW Apply wait logic
#                 if frame_num == 0: 
#                     cv2.putText(image, 'STARTING COLLECTION', (120,200), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     # Show to screen
#                     cv2.imshow('OpenCV Feed', image)
#                     cv2.waitKey(2000)
#                 else: 
#                     cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     # Show to screen
#                     cv2.imshow('OpenCV Feed', image)
                
# #                 NEW Export keypoints
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)
                
#                 # Break gracefully
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
                    
#     cap.release()
#     cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [98]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [99]:
DATA_PATH

'MP_Data_60FPS'

In [100]:
label_map = {label:num for num, label in enumerate(actions)}

In [101]:
label_map

{'Walk_': 0,
 'Wait_': 1,
 'Hello_L': 2,
 'Hello_R': 3,
 'Name_': 4,
 'No_': 5,
 'Sorry_R': 6,
 'Thanks_L': 7,
 'Fire_': 8,
 'Give up_': 9,
 'House_': 10,
 'Ready_': 11,
 'Drink_': 12,
 'Sleep_': 13,
 'Where_': 14}

In [102]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [103]:
X = np.array(sequences)
X.shape

(900, 30, 1662)

In [104]:
np.array(labels).shape
X.shape

(900, 30, 1662)

In [105]:
y = to_categorical(labels).astype(int)

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
y_test.shape

(90, 15)

In [107]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# 7. Build and Train LSTM Neural Network

In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard
from keras.optimizers import SGD

In [109]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [110]:
del model

In [111]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=(30,1662)))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
#model.add(Dense(actions.shape[0], activation='softmax'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [112]:
sgd = SGD(learning_rate=0.0005)

In [113]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [114]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
26/26 [==============================] - 7s 31ms/step - loss: 2.6887 - categorical_accuracy: 0.0938
Epoch 2/2000
26/26 [==============================] - 1s 24ms/step - loss: 2.4846 - categorical_accuracy: 0.1654
Epoch 3/2000
26/26 [==============================] - 1s 21ms/step - loss: 2.0977 - categorical_accuracy: 0.2605
Epoch 4/2000
26/26 [==============================] - 1s 24ms/step - loss: 1.8086 - categorical_accuracy: 0.3037
Epoch 5/2000
26/26 [==============================] - 1s 21ms/step - loss: 1.4950 - categorical_accuracy: 0.4432
Epoch 6/2000
26/26 [==============================] - 1s 25ms/step - loss: 1.3330 - categorical_accuracy: 0.4963
Epoch 7/2000
26/26 [==============================] - 1s 21ms/step - loss: 1.1567 - categorical_accuracy: 0.5716
Epoch 8/2000
26/26 [==============================] - 1s 22ms/step - loss: 0.8747 - categorical_accuracy: 0.6840
Epoch 9/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.7013 - categorical_

26/26 [==============================] - 0s 17ms/step - loss: 0.0953 - categorical_accuracy: 0.9630
Epoch 73/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0661 - categorical_accuracy: 0.9765
Epoch 74/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0519 - categorical_accuracy: 0.9802
Epoch 75/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0555 - categorical_accuracy: 0.9778
Epoch 76/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.0739 - categorical_accuracy: 0.9630
Epoch 77/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.1514 - categorical_accuracy: 0.9407
Epoch 78/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.1821 - categorical_accuracy: 0.9457
Epoch 79/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.0906 - categorical_accuracy: 0.9642
Epoch 80/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.1243 - categorical_accur

26/26 [==============================] - 0s 17ms/step - loss: 0.0371 - categorical_accuracy: 0.9877
Epoch 144/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0281 - categorical_accuracy: 0.9864
Epoch 145/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.0424 - categorical_accuracy: 0.9827
Epoch 146/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0702 - categorical_accuracy: 0.9704
Epoch 147/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.1400 - categorical_accuracy: 0.9469
Epoch 148/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0624 - categorical_accuracy: 0.9765
Epoch 149/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0247 - categorical_accuracy: 0.9926
Epoch 150/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0572 - categorical_accuracy: 0.9790
Epoch 151/2000
26/26 [==============================] - 1s 20ms/step - loss: 0.0630 - categoric

26/26 [==============================] - 0s 17ms/step - loss: 0.0104 - categorical_accuracy: 0.9963
Epoch 215/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0545 - categorical_accuracy: 0.9827
Epoch 216/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.2362 - categorical_accuracy: 0.9321
Epoch 217/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.1255 - categorical_accuracy: 0.9469
Epoch 218/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.0453 - categorical_accuracy: 0.9864
Epoch 219/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0266 - categorical_accuracy: 0.9901
Epoch 220/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0859 - categorical_accuracy: 0.9753
Epoch 221/2000
26/26 [==============================] - 1s 21ms/step - loss: 0.0750 - categorical_accuracy: 0.9790
Epoch 222/2000
26/26 [==============================] - 1s 19ms/step - loss: 0.0419 - categoric

26/26 [==============================] - 0s 16ms/step - loss: 0.0294 - categorical_accuracy: 0.9914
Epoch 286/2000
26/26 [==============================] - 0s 19ms/step - loss: 0.0132 - categorical_accuracy: 0.9963
Epoch 287/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0213 - categorical_accuracy: 0.9889
Epoch 288/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.0280 - categorical_accuracy: 0.9877
Epoch 289/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.0169 - categorical_accuracy: 0.9951
Epoch 290/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0128 - categorical_accuracy: 0.9963
Epoch 291/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0074 - categorical_accuracy: 0.9975
Epoch 292/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.0257 - categorical_accuracy: 0.9901
Epoch 293/2000
26/26 [==============================] - 0s 18ms/step - loss: 0.2832 - categoric

...

26/26 [==============================] - 0s 17ms/step - loss: 0.0064 - categorical_accuracy: 0.9988
Epoch 640/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 641/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0012 - categorical_accuracy: 1.0000
Epoch 642/2000
26/26 [==============================] - 0s 16ms/step - loss: 7.1874e-04 - categorical_accuracy: 1.0000
Epoch 643/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0021 - categorical_accuracy: 0.9988
Epoch 644/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0170 - categorical_accuracy: 0.9951
Epoch 645/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0026 - categorical_accuracy: 1.0000
Epoch 646/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0131 - categorical_accuracy: 0.9951
Epoch 647/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0072 - categ

26/26 [==============================] - 0s 19ms/step - loss: 0.3416 - categorical_accuracy: 0.9247
Epoch 710/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0278 - categorical_accuracy: 0.9938
Epoch 711/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0129 - categorical_accuracy: 0.9988
Epoch 712/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0132 - categorical_accuracy: 0.9951
Epoch 713/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0292 - categorical_accuracy: 0.9901
Epoch 714/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0090 - categorical_accuracy: 0.9975
Epoch 715/2000
26/26 [==============================] - 0s 16ms/step - loss: 0.0035 - categorical_accuracy: 1.0000
Epoch 716/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0021 - categorical_accuracy: 1.0000
Epoch 717/2000
26/26 [==============================] - 0s 17ms/step - loss: 0.0046 - categoric

26/26 [==============================] - 0s 16ms/step - loss: 3.2931e-04 - categorical_accuracy: 1.0000
Epoch 780/2000
26/26 [==============================] - 0s 16ms/step - loss: 7.6543e-04 - categorical_accuracy: 1.0000
Epoch 781/2000
26/26 [==============================] - 0s 17ms/step - loss: 4.1000e-04 - categorical_accuracy: 1.0000
Epoch 782/2000
26/26 [==============================] - 0s 17ms/step - loss: 4.0256e-04 - categorical_accuracy: 1.0000
Epoch 783/2000
26/26 [==============================] - 0s 17ms/step - loss: 9.3279e-04 - categorical_accuracy: 1.0000
Epoch 784/2000
26/26 [==============================] - 0s 16ms/step - loss: 2.1712e-04 - categorical_accuracy: 1.0000
Epoch 785/2000
26/26 [==============================] - 0s 17ms/step - loss: 1.5567e-04 - categorical_accuracy: 1.0000
Epoch 786/2000
26/26 [==============================] - 0s 16ms/step - loss: 1.6065e-04 - categorical_accuracy: 1.0000
Epoch 787/2000
26/26 [==============================] - 0s 17ms

Epoch 848/2000
26/26 [==============================] - 0s 17ms/step - loss: 2.2001e-05 - categorical_accuracy: 1.0000
Epoch 849/2000
26/26 [==============================] - 0s 16ms/step - loss: 1.1315e-04 - categorical_accuracy: 1.0000
Epoch 850/2000
26/26 [==============================] - 0s 16ms/step - loss: 8.9030e-05 - categorical_accuracy: 1.0000
Epoch 851/2000
26/26 [==============================] - 0s 16ms/step - loss: 2.6444e-05 - categorical_accuracy: 1.0000
Epoch 852/2000
26/26 [==============================] - 0s 16ms/step - loss: 2.6041e-05 - categorical_accuracy: 1.0000
Epoch 853/2000
26/26 [==============================] - 0s 16ms/step - loss: 6.0835e-05 - categorical_accuracy: 1.0000
Epoch 854/2000
26/26 [==============================] - 0s 17ms/step - loss: 5.2211e-05 - categorical_accuracy: 1.0000
Epoch 855/2000
26/26 [==============================] - 0s 17ms/step - loss: 2.5690e-05 - categorical_accuracy: 1.0000
Epoch 856/2000
26/26 [==========================


KeyboardInterrupt



In [118]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 dropout_3 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 dropout_4 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 64)               

In [21]:
#tensorboard --logdir=.

In [22]:
res = model.predict(X_test)

3/3 [==============================] - 1s 11ms/step


In [23]:
actions[np.argmax(res[1])]

'ready_'

In [24]:
actions[np.argmax(y_test[1])]

'ready_'

In [ ]:
res[14]

In [ ]:
np.argmax(res)

# 9 Save Weights

In [115]:
model.save('finalModel60FPSfinal14.h5')

In [ ]:
del model

In [ ]:
model.load_weights('finalModel60fpsfinal11.h5')

# 11 Test in real time

In [116]:
colors = [(0, 215, 255), (192,192,192), (50,127,205), (142, 241, 53), (123, 123 ,133), (214, 122, 141),(214, 122, 141),(214, 122, 141),
          (214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),(214, 122, 141),
         (245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141),(245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141),
         (245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141),(245,117,16), (117,245,16), (16,117,245), (142, 241, 53), (123, 123 ,133), (214, 122, 141)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    count = 0
    res_arr = np.array(res)
    idx = np.argsort(res_arr) 
    idx = idx[::-1]
    res_arr = res_arr[idx]
    new_actions = actions[idx]
#     for num, prob in enumerate(res):
#         cv2.rectangle(output_frame, (0,60+num*20), (int(prob*100), 90+num*20), colors[num], -1)
#         cv2.putText(output_frame, actions[num].split('_')[0], (0, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
#         cv2.putText(output_frame, str(int(prob*100)), (150, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
    
    cv2.putText(output_frame, 'Action', (0, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(output_frame, 'Percentage', (100, 65), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        
    for num, prob in enumerate(sorted(res, reverse= True)):
        count += 1
        cv2.rectangle(output_frame, (0,70+num*20), (int(prob*100), 90+num*20), colors[num], -1)
        cv2.putText(output_frame, new_actions[num].split('_')[0], (0, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(output_frame, str("{:.2f}".format(prob*100))+' %', (110, 85+num*20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
        if count == 3:
            break
       
    return output_frame


In [117]:
# 1. New detection variables
sequence = []
sentence = ['']
threshold = 0.75
predictions = []
no_of_frame = 0
a = 0
cap = cv2.VideoCapture(0)
# Set mediapipe model 
start_time = time.time()
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # Read feed
        ret, frame = cap.read()
        no_of_frame += 1
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
        
        #3. Viz logic
            if np.unique(predictions[-15:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    #if len(sentence) > 0: 
                        if actions[np.argmax(res)].split('_')[0] != sentence[-1]: 
                            if actions[np.argmax(res)].split('_')[0] == 'Wait' and sentence[-1]!='':#actions[0].split('_')[0]
                                sentence.append('') #actions[np.argmax(res)].split('_')[0]
                            #elif sentence[-1] == ' ':#actions[0].split('_')[0]
                            elif actions[np.argmax(res)].split('_')[0] != 'Wait' and sentence[-1]=='':
                                sentence.append(actions[np.argmax(res)].split('_')[0])
#                             elif np.unique(predictions[-90:])[0] == np.argmax(res) and a > 4 and actions[np.argmax(res)].split('_')[0] != 'wait':
#                                 sentence.append(actions[np.argmax(res)].split('_')[0])
                            
                   # else:
                       # sentence.append(actions[np.argmax(res)].split('_')[0])

            if len(sentence) > 7: 
                sentence = ['']

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
          
        #cv2.putText(image, "{}".format(no_of_frame), (600, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.rectangle(image, (0,0), (640, 40), (129, 196, 26), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        if no_of_frame == 30:
            no_of_frame = 0
            a += 1
        # Show to screen
        image = cv2.resize(image,(960,720))
        cv2.imshow('OpenCV Feed', image)
        
        # Break gracefully
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
end_time = time.time()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>

1/1 [==============================] - 0s 30ms/step
Where_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Where_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Where_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Where_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Where_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
Where_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Hello_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Hello_R
<class 'me

1/1 [==============================] - 0s 35ms/step
Thanks_L
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Thanks_L
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 36ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 35ms/step
Wait_
<class 'mediapip

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 23ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==================

...

1/1 [==============================] - 0s 34ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 30ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Ready_
<class 'media

1/1 [==============================] - 0s 35ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 25ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 46ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
Ready_
<class 'media

1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Ready_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 45ms/step
Ready_
<class 'media

1/1 [==============================] - 0s 44ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 34ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 38ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 72ms/step
Sorry_R
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Sorry_R
<cla

1/1 [==============================] - 0s 32ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 29ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 31ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 37ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 27ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 28ms/step
Wait_
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
Wait_
<class 'mediapipe.pyth

In [ ]:
np.unique(predictions[-30:])[0]

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
image = Image.open('B_L.jpg')

In [ ]:
arr_image = asarray(image)
arr_image.shape

In [ ]:
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    a, b = mediapipe_detection(cv2.cvtColor(arr_image, cv2.COLOR_BGR2RGB), holistic)

In [ ]:
draw_styled_landmarks(a, b)